In [1]:
import joblib
import numpy as np
from synctwin import io_utils

Constants used for generation

In [42]:
sim_id = '0.25_200'
seed = 509
model_id = ""

Load generated dataset

In [43]:
base_path_data = "data/pkpd/{}-seed-".format(sim_id) + str(seed)
data_path = base_path_data + "/{}-{}.{}"

# loading config and data
print("loading data")
n_units, n_treated, n_units_total, step, train_step, control_sample, noise, n_basis, n_cluster = io_utils.load_config(
    data_path, "train"
)
(
    x_full,
    t_full,
    mask_full,
    batch_ind_full,
    y_full,
    y_control,
    y_mask_full,
    m,
    sd,
    treatment_effect,
) = io_utils.load_tensor(data_path, "train")
(
    x_full_val,
    t_full_val,
    mask_full_val,
    batch_ind_full_val,
    y_full_val,
    y_control_val,
    y_mask_full_val,
    _,
    _,
    _,
) = io_utils.load_tensor(data_path, "val")

loading data


In [44]:
x = np.concatenate((x_full.cpu().numpy(), x_full_val.cpu().numpy()), axis=1)
t = np.concatenate((t_full.cpu().numpy(), t_full_val.cpu().numpy()), axis=1)
mask = np.concatenate((mask_full.cpu().numpy(), mask_full_val.cpu().numpy()), axis=1)
y = np.concatenate((y_full.cpu().numpy(), y_full_val.cpu().numpy()), axis=1)

In [45]:
(y_full[:, :10] == y_control[:, :10]).all()  # check which ones are the control

tensor(True, device='cuda:0')

Map to the convention used in our experiments

In [46]:
M = 5
H = 5
R = 5 
D = 3 
K = 1 
C = 3
X = x.transpose((1, 0, 2))
N = x.shape[0]
rand_index = np.random.permutation(N)
X = X[rand_index]
M_ = mask.transpose((1, 0, 2))[rand_index]
Y_pre = y.transpose((1, 0, 2))[rand_index]
Y_post = y.transpose((1, 0, 2))[rand_index]
A = np.concatenate((np.zeros((200, 1)), np.ones((200, 1)), np.zeros((200, 1)), np.ones((200, 1))), axis=0)[rand_index]
T = t.transpose((1, 0, 2))[rand_index]

In [47]:
N_train = round(N * 0.8)
N_test = round(N * 0.2)
X_train, X_test = X[:N_train], X[N_train:]
M_train, M_test = M_[:N_train], M_[N_train:]
Y_pre_train, Y_pre_test = Y_pre[:N_train], Y_pre[N_train:]
Y_post_train, Y_post_test = Y_post[:N_train], Y_post[N_train:]
A_train, A_test = A[:N_train], A[N_train:]
T_train, T_test = T[:N_train], T[N_train:]

In [48]:
joblib.dump((N_train, M, H, R, D, K, C, X_train, M_train, Y_pre_train, Y_post_train, A_train, T_train), f'data/pkpd/data_{sim_id}.joblib')
joblib.dump((N_test, M, H, R, D, K, C, X_test, M_test, Y_pre_test, Y_post_test, A_test, T_test), f'data/pkpd/test_data_{sim_id}.joblib')

['data/pkpd/test_data_0.25_200.joblib']